In [1]:
"""
Randomized measurements can estimate Rényi-2 entropy using random local Clifford rotations and repeated shots.

Exercises:
- Replace the analytic probabilities with sampled bitstrings (multinomial) to mimic finite shots.
- Compare a product state vs. Bell state.
"""

'\nRandomized measurements can estimate Rényi-2 entropy using random local Clifford rotations and repeated shots.\n\nExercises:\n- Replace the analytic probabilities with sampled bitstrings (multinomial) to mimic finite shots.\n- Compare a product state vs. Bell state.\n'

In [8]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, DensityMatrix, partial_trace
rng = np.random.default_rng(7)

def random_unitary_for_1q():
    # A random unitary on 1 qubit via Euler angles
    a,b,c = rng.random(3)*2*np.pi
    qc = QuantumCircuit(1)
    qc.rx(a,0); qc.ry(b,0); qc.rz(c,0)
    return qc

def renyi2_from_probs(p):
    return -np.log2(np.sum(p**2))

prep = QuantumCircuit(2)
prep.h(0); prep.cx(0,1)

M = 200
true_entropy = []
vals = []
for _ in range(M):
    #Apply random unitaries
    Ua, Ub = random_unitary_for_1q(), random_unitary_for_1q()
    circ = prep.compose(Ua, qubits=[0]).compose(Ub, qubits=[1])

    #Compute true entropy
    psi = Statevector.from_instruction(circ)
    rhoA = partial_trace(DensityMatrix(psi), [1]).data # Trace out qubit 1
    evals = np.linalg.eigvalsh(rhoA)
    true_entropy.append(renyi2_from_probs(evals))

    #Compute toy model probabilities
    proba = np.abs(psi.data)**2
    proba_A = np.array([np.sum(proba[0::2]), np.sum(proba[1::2])])  # Marginalize over qubit 1
    proba_A /= np.sum(proba_A)  # Normalize in case of bad numerical precision
    X = 2*(proba_A[0]**2  - proba_A[0]*proba_A[1] + proba_A[1]**2) # Formula from DOI: 10.1126/science.aau4963 (Brydges–Elben–Roos–Zoller)
    vals.append(X)

print("True ⟨S2⟩ (toy):", np.mean(true_entropy))
print("Estimated ⟨S2⟩ (toy):", -np.log2(np.mean(vals)))

True ⟨S2⟩ (toy): 1.0000000000000007
Estimated ⟨S2⟩ (toy): 1.0
